In [105]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [106]:
#https://drive.google.com/file/d/1gIA9AC8wNx2kdepuP2Y7K5J7LnihRD1V/view?usp=sharing
downloaded = drive.CreateFile({'id':'1gIA9AC8wNx2kdepuP2Y7K5J7LnihRD1V'}) 
downloaded.GetContentFile('Ratings.csv') 
#https://drive.google.com/file/d/11duMozcad56OxTZtGrW3nY2Ir0hKU8ul/view?usp=sharing
downloaded = drive.CreateFile({'id':'11duMozcad56OxTZtGrW3nY2Ir0hKU8ul'}) 
downloaded.GetContentFile('Teleplay.csv')
#https://drive.google.com/file/d/1TFQWYDQYwBf3cEtVsU6BtUnUZrwGOreH/view?usp=sharing
downloaded = drive.CreateFile({'id':'1TFQWYDQYwBf3cEtVsU6BtUnUZrwGOreH'}) 
downloaded.GetContentFile('New_Teleplay.csv')

In [13]:
import pandas as pd
df=pd.read_csv('Teleplay.csv')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
v = TfidfVectorizer()
df=df.fillna("")
x = v.fit_transform(df['genre'])

#count_matrix = cv.fit_transform(x)
count_matrix = x

In [14]:
cosine_sim = cosine_similarity(count_matrix)

In [15]:
def get_title_from_index(index):
    return df[df.index == index]["name"].values[0]
def get_index_from_title(title):
    return df[df.name == title].index[0]

In [16]:
df.head()

,teleplay_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",long,1,9.37,200630
1,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51,9.25,114262
2,9253,Steins;Gate,"Sci-Fi, Thriller",medium,24,9.17,673572
3,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51,9.16,151266
4,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",medium,10,9.15,93351


In [22]:
dfr=pd.read_csv("Ratings.csv")
dfr.head()

,user_id,teleplay_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [23]:
user=dfr[dfr["user_id"]==53698]
required=user['rating'].max()
user_likes=user[user['rating']==required]

In [24]:
user_likes=pd.merge(user_likes,df[['teleplay_id','name','rating']],how='inner',left_on='teleplay_id',right_on='teleplay_id')
user_likes = user_likes.rename(columns = {'rating_x':'user rating','rating_y':'general rating'})
user_likes

,user_id,teleplay_id,user rating,name,general rating
0,53698,59,10,Chobits,7.57
1,53698,101,10,Air,7.48
2,53698,256,10,Hoshi no Koe,7.53
3,53698,433,10,"Kumo no Mukou, Yakusoku no Basho",7.89
4,53698,497,10,Yami to Boushi to Hon no Tabibito,6.73
5,53698,587,10,Hanbun no Tsuki ga Noboru Sora,7.69
6,53698,610,10,Popotan,6.46
7,53698,656,10,Air in Summer,7.47
8,53698,694,10,Canvas 2: Niji-iro no Sketch,6.91
9,53698,750,10,Binchou-tan,7


In [25]:
movies={}
for n in df['name']:
  movies[n]=0

In [26]:
for n in user_likes['name']:
  movie_user_likes = n
  movie_index = get_index_from_title(movie_user_likes)
  similar_movies = list(enumerate(cosine_sim[movie_index]))
  sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
  i=0
  for element in sorted_similar_movies:
      movies[get_title_from_index(element[0])]+=1
      print(get_title_from_index(element[0]),end = " ")
      i=i+1
      if i>5:
          break

Boku no Marie Yume de Aetara Asatte Dance Seishun Fuufu Monogatari: Koiko no Mainichi Chobits Recap Kagaku na Yatsura Sola Kanon Kanon Kazahana Clannad: After Story Glasslip Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. Densetsu Kyojin Ideon: Hatsudou-hen Densetsu Kyojin Ideon: Sesshoku-hen The Humanoid: Ai no Wakusei Lezeria Dragonaut: The Resonance Special Captain Earth Macross Plus Uchuu Senkan Yamato: Aratanaru Tabidachi Soliton no Akuma Mobile Suit Gundam: The 08th MS Team - Miller&#039;s Report Tactical Roar Mahoromatic 2 Mobile Suit Gundam: The 08th MS Team Valkyrie Drive: Mermaid Valkyrie Drive: Mermaid Specials Gokujo.: Souda Onsen ni Ikou!! Patalliro Saiyuuki! Patalliro Saiyuuki! Special Uragiri wa Boku no Namae wo Shitteiru Maison Ikkoku: Kanketsu-hen Rec: Yurusarezarumono Maison Ikkoku: Ikkokujima Nanpa Shimatsuki Maison Ikkoku: Utsuriyuku Kisetsu no Naka de Maison Ikkoku: Prelude - Meguru Haru no Sakura no you ni... Chocotto Sister Sono Kinisasete yo: My My Mai 

In [27]:
recommended = sorted(movies.items(), key=lambda x:x[1],reverse=True)[:100]
print("The top 5 movies recommended are : ")
i=0
for l,k in recommended:
  if l not in list(user_likes['name']):
      print(l)
      i+=1
  if i==5:
    break

The top 5 movies recommended are : 
Mankitsu Happening
Koiito Kinenbi The Animation
Stringendo: Angel-tachi no Private Lesson
Fella Pure: Mitarashi-san Chi no Jijou The Animation
Sola


Collaborative Filtering

In [5]:
dfr['rating'].replace(-1,0,inplace=True)
dfc=pd.merge(dfr,df[['teleplay_id','name']],how='inner',left_on='teleplay_id',right_on='teleplay_id')
dfc=dfc.drop('teleplay_id',axis=1)
dfc.head()

,user_id,rating,name
0,1,0,Naruto
1,3,8,Naruto
2,5,6,Naruto
3,6,0,Naruto
4,10,0,Naruto


In [6]:
movie_matrix_UII = dfc.pivot_table(index='user_id', columns='name', values='rating')
movie_matrix_UII.head()

name,&quot;0&quot;,"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,&quot;Eiji&quot;,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Roots,.hack//Sign,.hack//The Movie: Sekai no Mukou ni,.hack//Unison,.hack//Versus: The Thanatos Report,0-sen Hayato,0-sen Hayato Pilot,001,009 Re:Cyborg,009-1,009-1: R&amp;B,00:08,07-Ghost,1+2=Paradise,100%,100-man-nen Chikyuu no Tabi: Bander Book,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,11eyes,11eyes Picture Drama,11eyes: Momoiro Genmutan,12-gatsu no Uta,12-sai.,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,15 Sonyeon Uju Pyoryugi,...,"Zone of the Enders: Dolores, I",Zone of the Enders: Idolo,Zonmi-chan: Halloween☆Special Movie!,Zoobles!,Zou no Inai Doubutsuen,Zou no Senaka: Tabidatsu Hi,Zouressha ga Yatte Kita,Zukkoke Sannin-gumi: Zukkoke Jikuu Bouken,Zumomo to Nupepe,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,_Summer,a_caFe,eX-Driver,eX-Driver the Movie,eX-Driver the Movie Specials,eX-Driver: Nina &amp; Rei Danger Zone,ef: A Tale of Melodies.,ef: A Tale of Melodies. - Prologue,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,ef: A Tale of Memories. - Recollections,fake!fake!,gdgd Fairies,gdgd Fairies 2,gdgd Fairies 2 Episode 0,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,himitsukesshatakanotsume.jp,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,lilac (bombs Jun Togawa),makemagic,"on-chan, Yume Power Daibouken!",s.CRY.ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
